# Supervised

This notebook extracts respiration signals with the following models:
- MTTS-CAN
- BigSmall

In [ ]:
import os
import numpy as np
import respiration.dataset as repository

dataset = repository.from_default()
scenarios = dataset.get_scenarios(['101_natural_lighting'])
scenarios

In [ ]:
results_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'pretrained')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

## Conduct experiments

In [ ]:
from datetime import datetime

evaluation_metadata = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
}

In [ ]:
import respiration.utils as utils

device = utils.get_torch_device()

In [ ]:
from respiration.extractor.mtts_can import (
    load_model,
    calculate_cutoff,
    preprocess_video_frames,
)
import respiration.extractor.big_small as big_small

extracted_signals = []

frame_depth = 10
mtts_model = load_model(frame_depth=frame_depth)
big_small_model, _ = big_small.load_model(device=device)

for (subject, setting) in scenarios:
    print(f'Processing {subject} - {setting}')

    frames, meta = dataset.get_video_rgb(subject, setting)

    #
    # MTTS-CAN model to extract the signal
    #

    mtts_start = datetime.now()

    resized, normalized = preprocess_video_frames(frames)
    cutoff = calculate_cutoff(resized.shape[0], frame_depth)

    resized = resized[:cutoff]
    normalized = normalized[:cutoff]

    mtts_raw = mtts_model.predict(
        (resized, normalized),
        batch_size=100
    )
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'model': 'mtts_can',
        'duration': datetime.now() - mtts_start,
        'sample_rate': meta.fps,
        'signal': np.cumsum(mtts_raw[1]).tolist(),
    })

    #
    # Use the BigSmall model to extract the signal
    #
    big_small_start = datetime.now()
    big_small_raw = big_small.batch_process(big_small_model, frames, device)
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'model': 'big_small',
        'duration': datetime.now() - big_small_start,
        'sample_rate': meta.fps,
        'signal': big_small_raw.tolist(),
    })

    # Garbage collect the frames
    del frames

In [ ]:
evaluation_metadata['timestamp_finish'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
# Save the extracted_signals as a JSON
json_path = os.path.join(results_dir, 'predictions.json')
utils.write_json(json_path, extracted_signals)

# Save the evaluation dataframe
csv_path = os.path.join(results_dir, 'predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(results_dir, 'parameters.json')
utils.write_json(json_path, evaluation_metadata)